<a href="https://colab.research.google.com/github/DarleneJD/ACOPF/blob/main/13barrasOPF_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MT

In [294]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [295]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [296]:
# if not (shutil.which("glpk") or os.path.isfile("glpk")):
#     if "google.colab" in sys.modules:
#         !apt-get install -y -qq glpk-utils
#     else:
#         try:
#             !conda install -c conda-forge glpk
#         except:
#             pass

In [297]:
!git clone https://github.com/DarleneJD/ACOPF.git

fatal: destination path 'ACOPF' already exists and is not an empty directory.


In [298]:
import math
from pathlib import Path
import pandas as pd
import pyomo.environ as pyo
from collections import defaultdict
# Solver (HiGHS)
from pyomo.environ import *
import pandas as pd
import numpy as np


In [299]:
!git clone https://github.com/DarleneJD/ACOPF.git


fatal: destination path 'ACOPF' already exists and is not an empty directory.


In [300]:
base_path = "/content/ACOPF"


In [301]:
branches_path = f"{base_path}/branches_1.xlsx"
buses_path    = f"{base_path}/buses_1.xlsx"

df_branches = pd.read_excel(branches_path, sheet_name="MT")
df_trafos   = pd.read_excel(branches_path, sheet_name="Trafos")
df_buses    = pd.read_excel(buses_path,    sheet_name="MT")


In [302]:
def normalize_phases(value):
    """
    Converte entradas como:
    1
    1.0
    '1,2,3'
    '1.0,2.0,3.0'
    '1.2.3'
    em lista de inteiros [1], [1,2,3]
    """
    if pd.isna(value):
        return []

    s = str(value).strip()

    # unifica separadores
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    phases = []
    for p in parts:
        try:
            ph = int(float(p))
            if ph in [1, 2, 3]:
                phases.append(ph)
        except ValueError:
            pass

    return sorted(set(phases))

# Branches (MT)
df_branches["phase_list"] = df_branches["phase"].apply(normalize_phases)

# Trafos
df_trafos["phase_list"] = df_trafos["phases"].apply(normalize_phases)


In [303]:
# Remover SourceBus
df_buses = df_buses[~df_buses["name"].astype(str).str.contains("Source", case=False)]

# Mapeamento name -> N (id numérico)
bus_id_map = dict(zip(df_buses["name"], df_buses["N"]))

# Lista final de barras MT
buses_mt = df_buses["N"].astype(int).unique().tolist()


In [304]:
# Converter barras l,k para identificador numérico
df_branches["from_bus"] = df_branches["l"].map(bus_id_map)
df_branches["to_bus"]   = df_branches["k"].map(bus_id_map)

# Remover ramos mal mapeados
df_branches = df_branches.dropna(subset=["from_bus", "to_bus"])

df_branches["from_bus"] = df_branches["from_bus"].astype(int)
df_branches["to_bus"]   = df_branches["to_bus"].astype(int)
df_branches["m"]        = df_branches["m"].astype(int)

# Conjunto de ramos MT
branches_mt = df_branches["m"].unique().tolist()


In [305]:
PHASES = [1, 2, 3]

branch_phase = []

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))  # garante inteiro

    raw = row["phase"]

    if pd.isna(raw):
        continue

    s = str(raw).strip()

    # Normalização:
    # troca qualquer separador por vírgula
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    for p in parts:
        # extrai apenas números válidos
        try:
            ph = int(float(p))
        except ValueError:
            continue

        if ph in PHASES:
            branch_phase.append((m, ph))

branch_phase = sorted(set(branch_phase))


In [306]:
R = {}
X = {}
Imax = {}
from_bus = {}
to_bus = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))  # robusto se vier 1.0

    from_bus[m] = int(row["from_bus"])
    to_bus[m]   = int(row["to_bus"])
    Imax[m]     = float(row["Imax"])

    # USA A COLUNA NORMALIZADA
    for ph in row["phase_list"]:
        R[(m, ph)] = float(row["R"])
        X[(m, ph)] = float(row["X"])



Length = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))
    Length[m] = float(row["Length_m"])

Vbase = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))
    fb = from_bus[m]
    Vbase[m] = float(df_buses.loc[df_buses["N"] == fb, "v_nom_kv"].iloc[0])
Smax = {}

for m in branches_mt:
    Smax[m] = Vbase[m] * Imax[m]  # kVA por fase


In [307]:
# Padronizar mv_bus
df_trafos["mv_bus_num"] = df_trafos["mv_bus"].map(bus_id_map)
df_trafos = df_trafos.dropna(subset=["mv_bus_num"])
df_trafos["mv_bus_num"] = df_trafos["mv_bus_num"].astype(int)

# Placeholder de potência vinda da BT
P_TR = {}
Q_TR = {}

for _, row in df_trafos.iterrows():
    mvb = int(row["mv_bus_num"])

    # USA DIRETAMENTE A LISTA NORMALIZADA
    for ph in row["phase_list"]:
        P_TR[(mvb, ph)] = 0.0
        Q_TR[(mvb, ph)] = 0.0


In [308]:
trafo_mv_bus = {
    row["trafo_id"]: int(row["mv_bus_num"])
    for _, row in df_trafos.iterrows()
}


alpha_tr = {}
beta_tr  = {}

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    r_per = float(row["r_per"]) / 100.0
    xhl   = float(row["XHL"]) / 100.0

    for ph in row["phase_list"]:
        alpha_tr[(tid, ph)] = r_per
        beta_tr[(tid, ph)]  = xhl


In [309]:
model = ConcreteModel()

# Conjuntos
model.BUS = Set(initialize=buses_mt)

bus_phase = set()

for (br, ph) in branch_phase:
    bus_phase.add((from_bus[br], ph))
    bus_phase.add((to_bus[br], ph))

model.BUS_PH = Set(dimen=2, initialize=bus_phase)


model.PH  = Set(initialize=PHASES)
model.BR  = Set(initialize=branches_mt)
model.BR_PH = Set(initialize=branch_phase, dimen=2)

trafo_phase = []

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    for ph in row["phase_list"]:
        trafo_phase.append((tid, ph))

model.TR_PH = Set(dimen=2, initialize=trafo_phase)


# Parâmetros
model.R = Param(model.BR_PH, initialize=R)
model.X = Param(model.BR_PH, initialize=X)

model.from_bus = Param(model.BR, initialize=from_bus)
model.to_bus   = Param(model.BR, initialize=to_bus)



# Variáveis
model.P = Var(model.BR_PH)
model.Q = Var(model.BR_PH)
model.v = Var(model.BUS, model.PH, bounds=(0.95**2, 1.05**2))

model.Ppos = Var(model.BR_PH, domain=NonNegativeReals)
model.Pneg = Var(model.BR_PH, domain=NonNegativeReals)
model.Qpos = Var(model.BR_PH, domain=NonNegativeReals)
model.Qneg = Var(model.BR_PH, domain=NonNegativeReals)
model.Ploss = Var(model.BR_PH, domain=NonNegativeReals)




In [310]:
# Potência aparente "equivalente" (L1) por trafo e fase
model.Str_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Str_neg = Var(model.TR_PH, domain=NonNegativeReals)
model.Ploss_tr = Var(model.TR_PH, domain=NonNegativeReals)
model.Qloss_tr = Var(model.TR_PH, domain=NonNegativeReals)

model.Ptr_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Ptr_neg = Var(model.TR_PH, domain=NonNegativeReals)
model.Qtr_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Qtr_neg = Var(model.TR_PH, domain=NonNegativeReals)




In [311]:
def Ptr_def(m, tid, ph):
    return m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph]

def Qtr_def(m, tid, ph):
    return m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph]


In [312]:
def voltage_drop_rule(m, br, ph):
    i = m.from_bus[br]
    j = m.to_bus[br]
    return m.v[j,ph] == m.v[i,ph] - 2*(
        (m.R[br,ph]*Length[br]) * m.P[br,ph] +
        (m.X[br,ph]*Length[br]) * m.Q[br,ph]
    )



In [313]:
def Ploss_rule(m, br, ph):
    return m.Ploss[br,ph] >= (
        m.R[br,ph] *
        (m.Ppos[br,ph] + m.Pneg[br,ph]
         + m.Qpos[br,ph] + m.Qneg[br,ph])
    )

model.PlossDef = Constraint(model.BR_PH, rule=Ploss_rule)


In [314]:
def active_balance_rule(m, bus, ph):

    inflow = sum(
        m.P[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.to_bus[br] == bus
    )

    outflow = sum(
        m.P[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.from_bus[br] == bus
    )

    trafo_power = sum(
        m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph] + m.Ploss_tr[tid,ph]
        for (tid,p) in m.TR_PH
        if p == ph and trafo_mv_bus[tid] == bus
    )

    return inflow - outflow == trafo_power

def reactive_balance_rule(m, bus, ph):

    inflow = sum(
        m.Q[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.to_bus[br] == bus
    )

    outflow = sum(
        m.Q[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.from_bus[br] == bus
    )

    trafo_power = sum(
        m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph] + m.Qloss_tr[tid,ph]
        for (tid,p) in m.TR_PH
        if p == ph and trafo_mv_bus[tid] == bus
    )

    return inflow - outflow == trafo_power

model.PBalance = Constraint(model.BUS_PH, rule=active_balance_rule)
model.QBalance = Constraint(model.BUS_PH, rule=reactive_balance_rule)


In [315]:
Vmin = 0.95  # pu

def current_limit_rule(m, br, ph):
    return (
        m.Ppos[br,ph] + m.Pneg[br,ph]
        + m.Qpos[br,ph] + m.Qneg[br,ph]
        <= Smax[br] * Vmin
    )

model.CurrentLimit = Constraint(model.BR_PH, rule=current_limit_rule)



In [316]:
slack_bus = int(df_buses[df_buses["tb"] == 3]["N"].iloc[0])

for ph in PHASES:
    model.v[slack_bus, ph].fix(1.0)


In [317]:
def Ploss_tr_rule(m, tid, ph):
    return m.Ploss_tr[tid,ph] >= alpha_tr[(tid,ph)] * (
        m.Ptr_pos[tid,ph] + m.Ptr_neg[tid,ph]
        + m.Qtr_pos[tid,ph] + m.Qtr_neg[tid,ph]
    )

model.PlossTrDef = Constraint(model.TR_PH, rule=Ploss_tr_rule)


def Qloss_tr_rule(m, tid,ph):
    return m.Qloss_tr[tid,ph] >= beta_tr[(tid,ph)] * (
        m.Str_pos[tid,ph] + m.Str_neg[tid,ph]
        + m.Qtr_pos[tid,ph] + m.Qtr_neg[tid,ph]
    )

model.QlossTrDef = Constraint(model.TR_PH, rule=Qloss_tr_rule)



In [318]:


model.obj = Objective(expr=0.0)


In [319]:
solver = SolverFactory("highs")
result = solver.solve(model, tee=True)

print(result.solver.termination_condition)


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 170 rows; 371 cols; 629 nonzeros
Coefficient ranges:
  Matrix  [6e-03, 1e+03]
  Cost    [0e+00, 0e+00]
  Bound   [0e+00, 0e+00]
  RHS     [8e+02, 3e+03]
Presolving model
61 rows, 151 cols, 284 nonzeros  0s
26 rows, 80 cols, 135 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-170); columns 0(-371); nonzeros 0(-629) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Objective value     :  0.0000000000e+00
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.01
optimal


# BT

In [320]:
# --- Ler BT e PV (mesmo branches_1.xlsx / buses_1.xlsx) ---
df_branches_bt = pd.read_excel(branches_path, sheet_name="BT")
df_buses_bt    = pd.read_excel(buses_path,    sheet_name="BT")
df_pv          = pd.read_excel(buses_path,    sheet_name="PV")  # PV está em buses_1.xlsx

# Normalizar fases em BT e PV
df_branches_bt["phase_list"] = df_branches_bt["phase"].apply(normalize_phases)
df_buses_bt["phase_list"]    = df_buses_bt["phases"].apply(normalize_phases)
df_pv["phase_list"]          = df_pv["phases"].apply(normalize_phases)

# Map name -> N na BT
bus_id_map_bt = dict(zip(df_buses_bt["name"], df_buses_bt["N"]))

buses_bt = df_buses_bt["N"].astype(int).unique().tolist()


In [321]:
# Mapear ramos BT: l,k (nomes) -> N
df_branches_bt["from_bus"] = df_branches_bt["l"].map(bus_id_map_bt)
df_branches_bt["to_bus"]   = df_branches_bt["k"].map(bus_id_map_bt)
df_branches_bt = df_branches_bt.dropna(subset=["from_bus", "to_bus"])

df_branches_bt["from_bus"] = df_branches_bt["from_bus"].astype(int)
df_branches_bt["to_bus"]   = df_branches_bt["to_bus"].astype(int)
df_branches_bt["m"]        = df_branches_bt["m"].astype(int)

branches_bt = df_branches_bt["m"].unique().tolist()

# Ramos por fase (BT)
branch_phase_bt = []
for _, row in df_branches_bt.iterrows():
    m = int(float(row["m"]))
    for ph in row["phase_list"]:
        branch_phase_bt.append((m, ph))
branch_phase_bt = sorted(set(branch_phase_bt))

# Barras por fase (BT)
bus_phase_bt = set()
for (br, ph) in branch_phase_bt:
    fb = int(df_branches_bt.loc[df_branches_bt["m"] == br, "from_bus"].iloc[0])
    tb = int(df_branches_bt.loc[df_branches_bt["m"] == br, "to_bus"].iloc[0])
    bus_phase_bt.add((fb, ph))
    bus_phase_bt.add((tb, ph))
bus_phase_bt = sorted(bus_phase_bt)


In [323]:
R_bt = {}
X_bt = {}
Imax_bt = {}
from_bus_bt = {}
to_bus_bt = {}
Length_bt_km = {}

for _, row in df_branches_bt.iterrows():
    m = int(float(row["m"]))
    from_bus_bt[m] = int(row["from_bus"])
    to_bus_bt[m]   = int(row["to_bus"])
    Imax_bt[m]     = float(row["Imax"])

    Lkm = float(row["Length_m"]) / 1000.0
    Length_bt_km[m] = Lkm

    for ph in row["phase_list"]:
        # R,X em ohm/km -> total em ohm
        R_bt[(m, ph)] = float(row["R"]) * Lkm
        X_bt[(m, ph)] = float(row["X"]) * Lkm

# Vbase por ramo BT (kV da barra de origem)
Vbase_bt = {}
for m in branches_bt:
    fb = from_bus_bt[m]
    Vbase_bt[m] = float(df_buses_bt.loc[df_buses_bt["N"] == fb, "v_nom_kv"].iloc[0])

# Smax BT (kVA por fase) para limite linear de corrente
Smax_bt = {m: Vbase_bt[m] * Imax_bt[m] for m in branches_bt}


In [324]:
Pload_bt = {}
Qload_bt = {}

for _, row in df_buses_bt.iterrows():
    b = int(row["N"])
    phs = row["phase_list"]
    if len(phs) == 0:
        continue

    Pd = float(row["P_D"]) if not pd.isna(row["P_D"]) else 0.0
    Qd = float(row["Q_D"]) if not pd.isna(row["Q_D"]) else 0.0

    share = 1.0 / len(phs)

    for ph in phs:
        Pload_bt[(b, ph)] = Pd * share
        Qload_bt[(b, ph)] = Qd * share


In [325]:
# PV por barra-fase
Ppv_bt = {}     # param (injeção ativa)
Qpv_max = {}    # param (limite reativo)

# Inicializa com zero para todas as barras/fases (se não houver PV)
for (b, ph) in bus_phase_bt:
    Ppv_bt[(b, ph)] = 0.0
    Qpv_max[(b, ph)] = 0.0

for _, row in df_pv.iterrows():
    bus_name = row["Bus"]
    if bus_name not in bus_id_map_bt:
        continue
    b = int(bus_id_map_bt[bus_name])

    # para cada fase, lê colunas p_pv_x e q_pv_x
    for ph in row["phase_list"]:
        pcol = f"p_pv_{ph}"
        qcol = f"q_pv_{ph}"
        Ppv_bt[(b, ph)] = float(row[pcol]) if pcol in row and not pd.isna(row[pcol]) else 0.0
        Qpv_max[(b, ph)] = float(row[qcol]) if qcol in row and not pd.isna(row[qcol]) else 0.0


In [326]:
# Mapear lv_bus (nome) -> N BT
df_trafos["lv_bus_num"] = df_trafos["lv_bus"].map(bus_id_map_bt)
df_trafos = df_trafos.dropna(subset=["lv_bus_num"])
df_trafos["lv_bus_num"] = df_trafos["lv_bus_num"].astype(int)

trafo_lv_bus = {row["trafo_id"]: int(row["lv_bus_num"]) for _, row in df_trafos.iterrows()}


In [327]:
# Conjuntos BT
model.BUS_BT = Set(initialize=buses_bt)
model.BUS_PH_BT = Set(dimen=2, initialize=bus_phase_bt)

model.BR_BT = Set(initialize=branches_bt)
model.BR_PH_BT = Set(dimen=2, initialize=branch_phase_bt)

# Parâmetros BT
model.R_BT = Param(model.BR_PH_BT, initialize=R_bt)
model.X_BT = Param(model.BR_PH_BT, initialize=X_bt)

model.from_bus_bt = Param(model.BR_BT, initialize=from_bus_bt)
model.to_bus_bt   = Param(model.BR_BT, initialize=to_bus_bt)

model.Pload_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Pload_bt.get((b,ph), 0.0), default=0.0)
model.Qload_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Qload_bt.get((b,ph), 0.0), default=0.0)

model.Ppv_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Ppv_bt.get((b,ph), 0.0), default=0.0)
model.Qpv_max = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Qpv_max.get((b,ph), 0.0), default=0.0)


In [328]:
# Variáveis BT
model.P_BT = Var(model.BR_PH_BT)
model.Q_BT = Var(model.BR_PH_BT)
model.v_BT = Var(model.BUS_BT, model.PH, bounds=(0.95**2, 1.05**2))

# Decomposição abs BT
model.Ppos_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Pneg_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Qpos_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Qneg_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)

# PV: q variável por barra-fase (inicial 0), limitada por Qpv_max
def qpv_bounds(m, b, ph):
    qmax = value(m.Qpv_max[b,ph])
    return (-qmax, qmax)

model.Qpv = Var(model.BUS_BT, model.PH, bounds=qpv_bounds, initialize=0.0)

# PF como variável "rastreamento" (init 1). (acoplamento pf↔Q será feito depois via MILP/PWL)
model.pf_pv = Var(model.BUS_BT, model.PH, bounds=(0.0, 1.0), initialize=1.0)


In [329]:
def vdrop_bt_rule(m, br, ph):
    i = m.from_bus_bt[br]
    j = m.to_bus_bt[br]
    return m.v_BT[j,ph] == m.v_BT[i,ph] - 2*(m.R_BT[br,ph]*m.P_BT[br,ph] + m.X_BT[br,ph]*m.Q_BT[br,ph])

model.VdropBT = Constraint(model.BR_PH_BT, rule=vdrop_bt_rule)


In [330]:
def pbal_bt_rule(m, bus, ph):

    inflow = sum(m.P_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.to_bus_bt[br]==bus)
    outflow = sum(m.P_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.from_bus_bt[br]==bus)

    # Soma potência dos trafos cujo lv_bus é esta barra (pode haver mais de um)
    Ptr_here = sum(
        m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph]
        for (tid,p) in m.TR_PH
        if p==ph and trafo_lv_bus.get(tid, None) == bus
    )

    return inflow - outflow == m.Pload_bt[bus,ph] - m.Ppv_bt[bus,ph] - Ptr_here

model.PBalanceBT = Constraint(model.BUS_PH_BT, rule=pbal_bt_rule)


def qbal_bt_rule(m, bus, ph):

    inflow = sum(m.Q_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.to_bus_bt[br]==bus)
    outflow = sum(m.Q_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.from_bus_bt[br]==bus)

    Qtr_here = sum(
        m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph]
        for (tid,p) in m.TR_PH
        if p==ph and trafo_lv_bus.get(tid, None) == bus
    )

    return inflow - outflow == m.Qload_bt[bus,ph] - m.Qpv[bus,ph] - Qtr_here

model.QBalanceBT = Constraint(model.BUS_PH_BT, rule=qbal_bt_rule)


In [331]:
def Psplit_bt_rule(m, br, ph):
    return m.P_BT[br,ph] == m.Ppos_BT[br,ph] - m.Pneg_BT[br,ph]
def Qsplit_bt_rule(m, br, ph):
    return m.Q_BT[br,ph] == m.Qpos_BT[br,ph] - m.Qneg_BT[br,ph]

model.PsplitBT = Constraint(model.BR_PH_BT, rule=Psplit_bt_rule)
model.QsplitBT = Constraint(model.BR_PH_BT, rule=Qsplit_bt_rule)

Vmin_bt = 0.95

def current_limit_bt_rule(m, br, ph):
    return (m.Ppos_BT[br,ph] + m.Pneg_BT[br,ph] + m.Qpos_BT[br,ph] + m.Qneg_BT[br,ph]) <= (Smax_bt[br] * Vmin_bt)

model.CurrentLimitBT = Constraint(model.BR_PH_BT, rule=current_limit_bt_rule)


In [332]:
bt_slacks = df_buses_bt[df_buses_bt["tb"] == 3]["N"].astype(int).tolist()
if len(bt_slacks) > 0:
    bt_slack = bt_slacks[0]
    for ph in PHASES:
        model.v_BT[bt_slack, ph].fix(1.0)


# Regulador